## Load libraries

Load in the libraries that you need

In [1]:
from __future__ import division
import json
import re
import csv
import math

from itertools import izip_longest
from collections import Counter

from sqlalchemy import create_engine, MetaData, Table
import pandas as pd
import numpy as np
import numpy.ma as ma
import seaborn as sns
import matplotlib.pyplot as plt

## Run some code

In [2]:
db_url = "sqlite:///participants.db"
table_name = 'turkdemo'
data_column_name = 'datastring'

# boilerplace sqlalchemy setup
engine = create_engine(db_url)
metadata = MetaData()
metadata.bind = engine
table = Table(table_name, metadata, autoload=True)

# make a query and loop through
s = table.select()
rows = s.execute()

data = []
for row in rows:
    data.append(row[data_column_name])

# Now we have all participant datastrings in a list.
# Let's make it a bit easier to work with:

# parse each participant's datastring as json object
# and take the 'data' sub-object
data = [json.loads(part)['data'] for part in data if part is not None]

# insert uniqueid field into trialdata in case it wasn't added
# in experiment:
for part in data:
    for record in part:
#         print(record)
        if type(record['trialdata']) is list:

            record['trialdata'] = {record['trialdata'][0]:record['trialdata'][1]}
        record['trialdata']['uniqueid'] = record['uniqueid']

# flatten nested list so we just have a list of the trialdata recorded
# each time psiturk.recordTrialData(trialdata) was called.
def isNotNumber(s):
    try:
        float(s)
        return False
    except ValueError:
        return True

data = [record['trialdata'] for part in data for record in part]

# filter out fields that we dont want using isNotNumber function
filtered_data = [{k:v for (k,v) in part.items() if isNotNumber(k)} for part in data]

# Put all subjects' trial data into a dataframe object from the
# 'pandas' python library: one option among many for analysis
data_frame = pd.DataFrame(filtered_data)
data_frame.head()

,internal_node_id,key_press,listNumber,list_words,recalled_words,responses,rt,state,stimulus,time_elapsed,trialNumber,trial_index,trial_type,uniqueid,weights
0,0.0-0.0,NaN,NaN,NaN,NaN,"{""Q0"":""FRFR-092116_0"",""Q1"":""Andy""}",29856.0,NaN,NaN,29864.0,NaN,0.0,survey-text,debug82SI3M:debugJ7ZQB7,NaN
1,0.0-0.0,NaN,NaN,NaN,NaN,"{""Q0"":"""",""Q1"":""""}",1080.0,NaN,NaN,1089.0,NaN,0.0,survey-text,debug82SI3M:debugJ7ZQB7,NaN
2,0.0-0.0,NaN,NaN,NaN,NaN,"{""Q0"":""FRFR-092116_0"",""Q1"":""ANDY""}",24464.0,NaN,NaN,24472.0,NaN,0.0,survey-text,debug82SI3M:debugJ7ZQB7,NaN
3,0.0-0.0,32.0,NaN,NaN,NaN,NaN,5319.0,NaN,NaN,5320.0,NaN,0.0,text,debug82SI3M:debugJ7ZQB7,NaN
4,0.0-1.0,32.0,NaN,NaN,NaN,NaN,5577.0,NaN,NaN,11901.0,NaN,1.0,text,debug82SI3M:debugJ7ZQB7,NaN
